In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import pearsonr
import sys
from pathlib import Path

In [2]:
import pandas as pd
import numpy as np

folder = r'C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\EG Full Year\\'
files = glob.glob(folder + '*Bear*.txt')
files.sort()
print("Files found:", files)


df_all = pd.DataFrame(columns=['year', 'jan_avg', 'feb_avg', 'march_avg', 'april_avg', 'may_avg', 'june_avg',
                               'july_avg', 'aug_avg', 'sep_avg', 'oct_avg', 'nov_avg', 'dec_avg'])

for file in files:
    print(file)
    df = pd.read_table(file)
    i1 = df.index[df['Unnamed: 0'].eq('Average Count')]
    df = df[i1[0] + 1:]
    df = df[df['Week starting on...'].str.contains('checklist', na=False) == False]
    df.replace(0, np.nan, inplace=True)

    # Extract years from 'Week starting on...' column
    years = df['Week starting on...'].str.extract('(\d+)')[0].astype(float).dropna().unique().astype(int)    

    for year in years:
        # Filter data for the current year
        df_year = df[df['Week starting on...'].str.contains(str(year), na=False)] # Apply the fix here
        
        # Reshape data and calculate monthly averages
        df_melted = df_year.melt(id_vars=['Week starting on...'], var_name='date', value_name='count')
        df_melted['month'] = pd.to_numeric(df_melted['date'].str.extract('(\d+)/\d+')[0], errors='coerce').astype('Int64')
        monthly_avgs = df_melted.groupby('month')['count'].mean()

        # Store monthly averages in df_all
        df_all = pd.concat([df_all, pd.DataFrame({'year': [year],
                                                  'jan_avg': [monthly_avgs.get(1, np.nan)],
                                                  'feb_avg': [monthly_avgs.get(2, np.nan)],
                                                  'march_avg': [monthly_avgs.get(3, np.nan)],
                                                  'april_avg': [monthly_avgs.get(4, np.nan)],
                                                  'may_avg': [monthly_avgs.get(5, np.nan)],
                                                  'june_avg': [monthly_avgs.get(6, np.nan)],
                                                  'july_avg': [monthly_avgs.get(7, np.nan)],
                                                  'aug_avg': [monthly_avgs.get(8, np.nan)],
                                                  'sep_avg': [monthly_avgs.get(9, np.nan)],
                                                  'oct_avg': [monthly_avgs.get(10, np.nan)],
                                                  'nov_avg': [monthly_avgs.get(11, np.nan)],
                                                  'dec_avg': [monthly_avgs.get(12, np.nan)]})])
                                                

# Convert df_all to xarray
df_all_xr = df_all.set_index('year').to_xarray()
df_all_xr = df_all_xr.rename_dims(year='time')
df_all_xr = df_all_xr.rename({'year': 'time'})

print(df_all_xr)

Files found: ['C:\\Users\\u1355\\OneDrive\\Desktop\\Data Viz\\Thesis Website\\birds\\EG Full Year\\Eared Grebe Bear River 2004-2008.txt', 'C:\\Users\\u1355\\OneDrive\\Desktop\\Data Viz\\Thesis Website\\birds\\EG Full Year\\Eared Grebe Bear River 2009-2013.txt', 'C:\\Users\\u1355\\OneDrive\\Desktop\\Data Viz\\Thesis Website\\birds\\EG Full Year\\Eared Grebe Bear River 2014-2018.txt', 'C:\\Users\\u1355\\OneDrive\\Desktop\\Data Viz\\Thesis Website\\birds\\EG Full Year\\Eared Grebe Bear River 2019-2023.txt']
C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\EG Full Year\Eared Grebe Bear River 2004-2008.txt
C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\EG Full Year\Eared Grebe Bear River 2009-2013.txt
C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\EG Full Year\Eared Grebe Bear River 2014-2018.txt
C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\EG Full Year\Eared Grebe Bear River 2019-2023.txt


C:\Users\u1355\AppData\Local\Temp\ipykernel_17616\2975932823.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all, pd.DataFrame({'year': [year],


<xarray.Dataset> Size: 2kB
Dimensions:    (time: 20)
Coordinates:
  * time       (time) object 160B 2004 2005 2006 2007 ... 2020 2021 2022 2023
Data variables:
    jan_avg    (time) float64 160B nan nan nan nan nan ... 1.0 4.0 1.0 14.5 1.0
    feb_avg    (time) float64 160B nan nan nan nan nan ... nan nan nan nan nan
    march_avg  (time) float64 160B nan 161.0 nan 1.0 nan ... 4.778 2.8 2.333 nan
    april_avg  (time) float64 160B nan 1.872e+03 304.0 ... 25.87 16.81 13.3
    may_avg    (time) float64 160B nan nan 738.0 21.88 ... 18.13 27.63 5.223
    june_avg   (time) float64 160B nan 728.0 418.5 nan ... nan 1.667 1.418 4.458
    july_avg   (time) float64 160B nan nan 10.0 5.0 nan ... 2.0 9.375 1.25 4.667
    aug_avg    (time) float64 160B nan 347.0 2.0 4.5 ... 4.0 13.64 20.21 5.708
    sep_avg    (time) float64 160B nan 12.0 1.0 5.0 nan ... 3.75 23.5 4.067 5.0
    oct_avg    (time) float64 160B 1.0 623.5 18.5 nan ... 18.93 3.167 5.875
    nov_avg    (time) float64 160B 541.8 170.3 3.8

C:\Users\u1355\AppData\Local\Temp\ipykernel_17616\2975932823.py:52: UserWarning: rename 'year' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  df_all_xr = df_all_xr.rename({'year': 'time'})


In [4]:
df_all_df = df_all_xr.to_dataframe()

In [5]:
df_all_df

,jan_avg,feb_avg,march_avg,april_avg,may_avg,june_avg,july_avg,aug_avg,sep_avg,oct_avg,nov_avg,dec_avg
time,,,,,,,,,,,,
2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,541.750000,NaN
2005,NaN,NaN,161.000000,1871.500000,NaN,728.000000,NaN,347.000000,12.000000,623.500000,170.333333,NaN
2006,NaN,NaN,NaN,304.000000,738.000000,418.500000,10.000000,2.000000,1.000000,18.500000,3.833333,NaN
2007,NaN,NaN,1.000000,13.666667,21.875000,NaN,5.000000,4.500000,5.000000,NaN,20.000000,2.000000
2008,NaN,NaN,NaN,56.500000,3.000000,12.000000,NaN,NaN,NaN,1.000000,NaN,NaN
2009,NaN,NaN,1.000000,5.500000,3.750000,NaN,3.000000,NaN,11.000000,7.333333,19.000000,NaN
2010,1.000000,NaN,NaN,59.250000,11.666667,NaN,246.000000,NaN,12.000000,60.166667,15.633333,8.166667
2011,NaN,NaN,4.000000,83.208333,22.950000,44.500000,6.500000,15.000000,26.000000,48.000000,7.888889,NaN
2012,NaN,NaN,4.000000,1138.000000,108.200000,49.000000,11.500000,104.333333,52.066667,229.750000,35.500000,1.000000


In [7]:
df_all_df.to_json(r'C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\eg_geojsons\EG_Year_Avgs.json')